In [82]:
# Input a string pathway to a pip requirements.txt file or a conda environment.yml file. 
# You can remove this variable or leave it blank to have nb2app attempt to determine the dependencies automatically.
# Examples:
# requirements_file = 'requirements.txt'
# requirements_file = 'my/path/environment.yml'
requirements_file: str = '/Users/christopherpuglisi/Netrias/Projects/hyperparameter_optimizer/src/environment.yml'
# Input string pathways to any files that the notebook will need to function when it is
# containerized. 
# You can delete/leave this empty if there are no external files to include. 
# Example:
# my_resource_files ['/absolute/path/encoders/my_encoder.h5', '/absolute/path/decoders/my_decoder.h5']
resource_files: list = [
]
from pathlib import Path
data_path: Path = Path('/Users/christopherpuglisi/Netrias/Projects/hyperparameter_optimizer/tests/aethr_m_subset_aethr_features_overlapped_02102025.pkl')
model_path: Path = Path('/Users/christopherpuglisi/Netrias/Projects/hyperparameter_optimizer/tests/best_model_LogisticClassifier_20250203_124517.pkl')

In [83]:
import pickle
import pandas as pd
import numpy as np
import joblib
from sklearn.metrics import roc_auc_score, accuracy_score
# Adjust the Python path to include the 'src' directory
import sys
import os
import ast
import warnings
import datetime
import json
from scipy.stats import zscore
warnings.filterwarnings("ignore")
src_path = os.path.abspath('../src')
if src_path not in sys.path:
    sys.path.append(src_path)
from models.knn_models import KNNClassifierModel
from optimizers.grid_search import GridSearchOptimizer
from loss_functions.factory import LossFunctionFactory
from models.bayesian_models import GaussianNBModel
from optimizers.random_search import RandomSearchOptimizer
from models.ensemble_models import RandomForestModel
from models.linear_models import LogisticRegression
from models.ensemble_models import RandomForestModel, GradientBoostingModel
from models.bayesian_models import GaussianNBModel, BernoulliNBModel
from models.knn_models import KNNClassifierModel
from models.svm_models import SVCModel
from models.bayesian_models import BayesianRidgeModel
from models.linear_models import LogisticModel, RidgeModel, LassoModel
from models.knn_models import KNNRegressorModel
from models.svm_models import SVRModel
from models.clustering_models import KMeansModel, DBSCANModel, AgglomerativeClusteringModel, SpectralClusteringModel
from sklearn.model_selection import train_test_split


In [19]:
data_cares: Path = Path('/Users/christopherpuglisi/Netrias/Projects/hyperparameter_optimizer/tests/cares_subset_aethr_features_overlapped_01312025.pkl')

In [20]:
with open(data_cares, 'rb') as f:
    data_cares = pickle.load(f)

X_c = data_cares.iloc[:,3:].fillna(0)
y_c = data_cares['Threat']

In [33]:
X_c.columns

Index(['A1A5D9', 'A6NDG6', 'O00233', 'O00299', 'O00391', 'O00410', 'O00442',
       'O00560', 'O00592', 'O00743',
       ...
       'Q15717', 'Q4V328', 'Q8N392', 'Q96CX2', 'Q96JP5', 'Q9BW30', 'Q9ULH7',
       'Q9Y4E1', 'Q9Y6H1', 'Q5T1J5'],
      dtype='object', length=561)

In [ ]:
data_path: Path = Path('/Users/christopherpuglisi/Netrias/Projects/hyperparameter_optimizer/tests/aethr_m_subset_aethr_features_overlapped_02102025.pkl')

In [33]:
raw_a_cov_path: Path = Path('/Users/christopherpuglisi/Netrias/Projects/cares/notebooks/AETHR_raw_7450_features_1125024.csv')

In [85]:
raw_a_cov = pd.read_pickle(data_path)

In [86]:
raw_a_cov

,Threat,FileName,SampleName,Donor,Strain,Time,Rep,State,Created,Comment,...,Q5T764,Q96L21,Q8NI22,Q9HAU6,G2XKQ0,P0DP23,Q11128,Q01081,P31997,Q5VTE0
0,1,AETHR_EPI_Don_1_EZ_120_A_P_Bane_06Aug24_BEH-Co...,AETHR_EPI_Don_1_EZ_120_A,1,EZ,120,A,Complete,8/6/2024 11:08 PM,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,AETHR_EPI_Don_1_EZ_120_B_P_Arwen_19Sep24_BEH-C...,AETHR_EPI_Don_1_EZ_120_B,1,EZ,120,B,Complete,9/21/2024 1:10 AM,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1,AETHR_EPI_Don_1_EZ_120_C_P_Bane_12Aug24_BEH-Co...,AETHR_EPI_Don_1_EZ_120_C,1,EZ,120,C,Complete,8/16/2024 11:24 AM,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1,AETHR_EPI_Don_1_EZ_120_D_P_Bane_01Sep24_BEH-Co...,AETHR_EPI_Don_1_EZ_120_D,1,EZ,120,D,Complete,9/2/2024 10:35 AM,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1,AETHR_EPI_Don_1_EZ_144_A_P_Bane_19Aug24_BEH-Co...,AETHR_EPI_Don_1_EZ_144_A,1,EZ,144,A,Complete,8/20/2024 5:59 AM,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
102,0,AETHR_EPI_Don_3_Mock_144_D_P_Bane_19Aug24_BEH-...,AETHR_EPI_Don_3_Mock_144_D,3,Mock,144,D,Complete,8/20/2024 9:02 AM,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
103,0,AETHR_EPI_Don_3_Mock_168_A_P_Bane_19Aug24_BEH-...,AETHR_EPI_Don_3_Mock_168_A,3,Mock,168,A,Complete,8/22/2024 4:16 AM,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
104,0,AETHR_EPI_Don_3_Mock_168_B_P_Bane_27Aug24_BEH-...,AETHR_EPI_Don_3_Mock_168_B,3,Mock,168,B,Complete,8/29/2024 4:40 AM,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
105,0,AETHR_EPI_Don_3_Mock_168_C_P_Bane_06Aug24_BEH-...,AETHR_EPI_Don_3_Mock_168_C,3,Mock,168,C,Complete,8/7/2024 2:25 PM,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [35]:
features_list = list(X_c.columns)

In [41]:
def match_and_rename_columns(X_c, raw_a_cov):
    features_list = list(X_c.columns)
    
    # Dictionary to store matched column names
    matched_columns = {}
    
    for col in raw_a_cov.columns:
        for feature in features_list:
            if feature in col:  # Check if feature is a substring in col
                matched_columns[col] = feature  # Map original column name to feature
                break  # Stop checking once a match is found
    
    # Keep only matched columns and rename them
    raw_a_cov_filtered = raw_a_cov[list(matched_columns.keys())].rename(columns=matched_columns)
    
    # Reorder to match features_list and add missing columns with 0s
    for feature in features_list:
        if feature not in raw_a_cov_filtered.columns:
            raw_a_cov_filtered[feature] = 0
    
    raw_a_cov_filtered = raw_a_cov_filtered[features_list]
    
    return raw_a_cov_filtered


In [45]:
def match_and_rename_columns(X_c, raw_a_cov):
    features_list = list(X_c.columns)
    
    # Dictionary to store matched column names
    matched_columns = {}
    
    for col in raw_a_cov.columns:
        for feature in features_list:
            if feature in col:  # Check if feature is a substring in col
                matched_columns[col] = feature  # Map original column name to feature
                break  # Stop checking once a match is found
    
    # Keep only matched columns and rename them
    raw_a_cov_filtered = raw_a_cov[list(matched_columns.keys())].rename(columns=matched_columns)
    
    # Reorder to match features_list and add missing columns with 0s
    for feature in features_list:
        if feature not in raw_a_cov_filtered.columns:
            raw_a_cov_filtered[feature] = 0
    
    raw_a_cov_filtered = raw_a_cov_filtered[features_list]
    
    # Add 'Unnamed: 0' column from raw_a_cov to raw_a_cov_filtered
    if 'Unnamed: 0' in raw_a_cov.columns:
        raw_a_cov_filtered.insert(0, 'Unnamed: 0', raw_a_cov['Unnamed: 0'])
    
    return raw_a_cov_filtered

In [46]:
matched_a_cov = match_and_rename_columns(X_c, raw_a_cov)

In [60]:
matched_a_cov_bulk = matched_a_cov[~matched_a_cov['Unnamed: 0'].str.contains('FACS')]

In [61]:
matched_a_cov_bulk.insert(0, 'Threat', 0)

# Set 'Threat' to 1 where 'Unnamed: 0' contains 'S2'
matched_a_cov_bulk.loc[matched_a_cov_bulk['Unnamed: 0'].str.contains('S2', na=False), 'Threat'] = 1

In [62]:
log2_normalized = np.log2(matched_a_cov_bulk.iloc[:, 2:] + 1)

# Apply z-score standardization
z_score_standardized = log2_normalized.apply(zscore, axis=0)

# Replace NaNs with 0
z_score_standardized = z_score_standardized.fillna(0)

# Update the original DataFrame
matched_a_cov_bulk.iloc[:, 2:] = z_score_standardized

In [63]:
matched_a_cov_bulk

,Threat,Unnamed: 0,A1A5D9,A6NDG6,O00233,O00299,O00391,O00410,O00442,O00560,...,Q15717,Q4V328,Q8N392,Q96CX2,Q96JP5,Q9BW30,Q9ULH7,Q9Y4E1,Q9Y6H1,Q5T1J5
4,0,Intensity AETHR_D1_MCK_96C_P_23Jan22_Smeagol_W...,0.0,0.0,0.085856,1.236387,0.0,-0.013334,0.300999,0.380349,...,1.273864,0.0,0.532450,0.0,0.0,0.843331,0.0,0.0,0.901261,0.0
10,1,Intensity AETHR_D1_S2_48E_P_23Jan22_Smeagol_WB...,0.0,0.0,-0.105198,-0.210603,0.0,0.255879,0.172733,-0.407778,...,-0.259957,0.0,-1.284848,0.0,0.0,-1.105007,0.0,0.0,0.005891,0.0
13,0,Intensity AETHR_D2_MCK_48C_P_23Jan22_Smeagol_W...,0.0,0.0,0.573652,-0.776181,0.0,-1.304550,-0.747083,0.037344,...,0.171515,0.0,-1.162726,0.0,0.0,-1.814221,0.0,0.0,-0.718667,0.0
17,0,Intensity AETHR_D1_MCK_72D_P_23Jan22_Smeagol_W...,0.0,0.0,-0.575744,0.170750,0.0,0.017483,0.397873,0.063429,...,0.318761,0.0,-0.877770,0.0,0.0,0.327521,0.0,0.0,0.577144,0.0
18,1,Intensity AETHR_D1_S2_48D_P_23Jan22_Smeagol_WB...,0.0,0.0,-0.832346,-0.695367,0.0,-1.300739,-3.386281,-1.109451,...,-0.520169,0.0,-0.518915,0.0,0.0,-0.206775,0.0,0.0,-2.035776,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
205,0,Intensity AETHR_D1_MCK_72B_P_23Jan22_Smeagol_W...,0.0,0.0,-0.681428,-1.441741,0.0,-1.313447,-1.052032,-1.660433,...,-1.212885,0.0,-0.677993,0.0,0.0,-0.344652,0.0,0.0,-2.757404,0.0
206,0,Intensity AETHR_D2_MCK_48D_P_23Jan22_Smeagol_W...,0.0,0.0,0.504529,-0.423948,0.0,-0.527250,-1.215335,0.051695,...,0.526651,0.0,-1.265530,0.0,0.0,-1.678024,0.0,0.0,0.328933,0.0
208,1,Intensity AETHR_D1_S2_48B_P_23Jan22_Smeagol_WB...,0.0,0.0,0.661054,0.144195,0.0,0.764824,0.688817,0.042705,...,0.239601,0.0,-0.904098,0.0,0.0,-0.648937,0.0,0.0,0.059618,0.0
213,0,Intensity AETHR_D1_MCK_72A_P_19Jan22_Smeagol_W...,0.0,0.0,-0.044270,1.315635,0.0,1.207161,1.372058,0.174786,...,1.094012,0.0,0.960403,0.0,0.0,1.048588,0.0,0.0,0.571644,0.0


In [64]:
X = matched_a_cov_bulk.iloc[:,2:]
y = matched_a_cov_bulk['Threat']

In [66]:
# Identify columns in X_c that are not in X
missing_columns = set(X_c.columns) - set(X.columns)

# Print the missing columns
print("Columns in X_c but not in X:", missing_columns)

Columns in X_c but not in X: set()


In [67]:
for col in missing_columns:
    X[col] = 0

In [39]:
X = X[X_c.columns]

In [68]:
model = joblib.load(model_path)

In [41]:
X_train, X_val, y_train, y_val = train_test_split(X_c, y_c, test_size=0.2, stratify=y_c, random_state=42)

In [42]:
aethr: Path = Path('/Users/christopherpuglisi/Netrias/Projects/hyperparameter_optimizer/tests/aethr_subset_aethr_features_overlapped_01312025.pkl')
with open(aethr, 'rb') as f:
    aethr = pickle.load(f)
aethr['Threat'] = (aethr['Virus_meta_col'] == 'S2').astype(int)
X_a = aethr.iloc[:,28:-1].fillna(0)
y_a = aethr['Threat']

In [54]:
num_models = 2500

# Store results
best_model = None
best_auc = 0
the_val_auc = 0
the_c_auc = 0
best_features = None
results = []

# Train multiple models with different feature subsets
for i in range(num_models):
    # Randomly select a feature subset
    num_features = np.random.randint(10, X_c.shape[1])  # Random number of features
    selected_features = np.random.choice(X_c.columns, num_features, replace=False)

    # Train logistic regression
    model = LogisticRegression(max_iter=1000, solver='liblinear', random_state=i)
    model.fit(X_train[selected_features], y_train)

    # Validate on internal validation set
    y_val_pred = model.predict_proba(X_val[selected_features])[:, 1]
    auc_val = roc_auc_score(y_val, y_val_pred)

    # Test on holdout set (X, y)
    y_holdout_pred = model.predict_proba(X_a[selected_features])[:, 1]
    auc_holdout = roc_auc_score(y_a, y_holdout_pred)

    y_holdout_pred_2 = model.predict_proba(X[selected_features])[:, 1]
    auc_holdout_2 = roc_auc_score(y, y_holdout_pred_2)

    # Store results
    results.append({
        'model_id': i,
        'num_features': num_features,
        'auc_val': auc_val,
        'auc_holdout': auc_holdout,
        'features': selected_features
    })

    auc_av = (auc_holdout + auc_holdout_2) / 2

    # Track the best model
    if auc_av > best_auc:
        the_val_auc = auc_val
        the_c_auc = auc_holdout_2
        the_m_auc = auc_holdout
        best_auc = auc_av
        best_model = model
        best_features = selected_features

# Convert results to DataFrame and display
results_df = pd.DataFrame(results)


In [55]:
print(f"The AUC on Val: {the_val_auc}")
print(f"Best AUC on Aethr Cov: {auc_holdout}")
print(f"The AUC on Aethr Mea: {auc_holdout_2}")
print(f"Best Feature Set ({len(best_features)} features):", best_features)

The AUC on Val: 0.9609929078014184
Best AUC on Aethr Cov: 0.791005291005291
The AUC on Aethr Mea: 0.6968253968253968
Best Feature Set (32 features): ['Q15717' 'Q9ULV0' 'Q12906' 'P84098' 'Q9Y4X5' 'P67870' 'P00167' 'P49821'
 'P50225' 'Q92696' 'P46060' 'Q8WXI7' 'Q96RN5' 'Q9BVK6' 'Q9HD33' 'Q15050'
 'Q8WU90' 'O14933' 'Q04323' 'Q8IZL8' 'Q2VIR3' 'P62861' 'Q99816' 'Q9H0P0'
 'P09661' 'O95436' 'Q02218' 'P80217' 'P31997' 'O14744' 'Q9P258' 'Q8IXQ6']


In [22]:
aethr: Path = Path('/Users/christopherpuglisi/Netrias/Projects/hyperparameter_optimizer/tests/aethr_subset_aethr_features_overlapped_01312025.pkl')
with open(aethr, 'rb') as f:
    aethr = pickle.load(f)
aethr['Threat'] = (aethr['Virus_meta_col'] == 'S2').astype(int)
X_c = aethr.iloc[:,28:-1].fillna(0)
y_c = aethr['Threat']

In [74]:
uniprot_ids = [
    'Q9Y265', 'Q9NR46', 'P09914', 'Q13421', 'P80188', 'O00233', 'P60763', 'P49406', 'Q9BR76', 'O14672',
    'P17936', 'Q7L5N1', 'Q15717', 'P30153', 'Q8NC56', 'P00167', 'Q96BP2', 'Q14694', 'A1A5D9', 'Q8IXQ6',
    'P78318', 'Q13310', 'P53007', 'P62070', 'P80217', 'Q9UJY5', 'P49821', 'O43865', 'P42224', 'Q96MU7',
    'Q9P258', 'Q96TA2', 'P39019', 'O95831', 'Q53GD3', 'Q9UIA9'
]

In [75]:
X_train, X_val, y_train, y_val = train_test_split(X_c, y_c, test_size=0.2, stratify=y_c, random_state=42)

In [78]:
num_features = np.random.randint(10, X_c.shape[1])  # Random number of features
selected_features = uniprot_ids

# Train logistic regression
model = LogisticRegression(max_iter=1000, solver='liblinear', random_state=0)
model.fit(X_train[selected_features], y_train)

# Validate on internal validation set
y_val_pred = model.predict_proba(X_val[selected_features])[:, 1]
auc_val = roc_auc_score(y_val, y_val_pred)

In [81]:
y_pred = model.predict(X[selected_features])

# Calculate accuracy
AUC = roc_auc_score(y, y_pred)

# Return accuracy
print(f"Model AUC: {AUC:.4f}")

accuracy = accuracy_score(y, y_pred)

# Return accuracy
print(f"Model accuracy: {accuracy:.4f}")

Model AUC: 0.5524
Model accuracy: 0.6000


In [70]:
best_features_47 = best_features.copy()

In [71]:
y_pred = model.predict(X)

# Calculate accuracy
AUC = roc_auc_score(y, y_pred)

# Return accuracy
print(f"Model AUC: {AUC:.4f}")

accuracy = accuracy_score(y, y_pred)

# Return accuracy
print(f"Model accuracy: {accuracy:.4f}")

Model AUC: 0.4968
Model accuracy: 0.5125
